## Trending topics

#### While topic modeling such as LDA seems to be an easy tool to identify topics, my past experience using it showed three drawbacks when it comes to short and high-frequency content.

1. Once an unsupervised model is selected, classfications become static, which means "unseen" topics in the future won't be able to be captured.

2. Short messages are more likely to contain a homogenous topic than not, which challenges LDA's assumption of a probability distribution of "multiple" topics in each document.

3. Ultimately LDA is a statistical model that isn't quite able capture similar semantics of words

#### Instead, here I build a *"live"* pipeline of trending topic detector combining several techniques such as Doc2Vec and hierachical clustering. This approach allws us to focus on recent documents, regardless what the past topic distribution might look like.


#### _Please note: here I don't have any API for news feed, thus using a file sent from a friend of mine. However the approach is similar._

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import xlrd
import matplotlib as plt

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer

from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering

import gensim

### 0. Load and Clean file

In [2]:
text_df = pd.read_csv('sample_file.csv') 
print(text_df.shape)
print(text_df.columns)

# remove rows that indicates "please ignore"
text_df = text_df[text_df.HEADLINE_ALERT_TEXT.str.contains("Test, Please Ignore")==False]
text_df.LANGUAGE.value_counts()

#### Only English
text_df = text_df[text_df['LANGUAGE']=='EN']

## Use Headline when no Take Text is available
text_df.loc[text_df['TAKE_TEXT'].isnull(),'TAKE_TEXT'] = text_df.loc[text_df['TAKE_TEXT'].isnull(),'HEADLINE_ALERT_TEXT']

## fill missing time
text_df.DATE = text_df.DATE.fillna(method = 'ffill')
text_df.TIME = text_df.TIME.fillna(method = 'ffill')

## format time

text_df.DATE = pd.to_datetime(text_df.DATE)
text_df['HOUR'] = text_df.TIME.apply(lambda x: x.split(':')[0])

# cleaning text
text_df.TAKE_TEXT = text_df.TAKE_TEXT.str.lower().str.strip().str.replace('[^\w\s]''',' ').str.replace('[^a-zA-Z0-9'' ]',' ').str.replace(r'\W*\b\w{1,1}\b', '')

text_df.TAKE_TEXT = text_df.TAKE_TEXT.apply(lambda x: re.sub(r'[\*|\+|\_|\-|\<||>|\(|\)]','',x))

text_df = text_df[text_df.TAKE_TEXT.notnull()]

# apply gensim processing
text_df['TAKE_TEXT'] = text_df['TAKE_TEXT'].apply(gensim.utils.simple_preprocess)
print(text_df.shape)

train_corpus = text_df.TAKE_TEXT.tolist()
train_corpus = [gensim.models.doc2vec.TaggedDocument(value, [key])for key , value in enumerate(train_corpus)]
len(train_corpus)

(8213, 19)
Index(['DATE', 'TIME', 'UNIQUE_STORY_INDEX', 'EVENT_TYPE', 'PNAC',
       'STORY_DATE_TIME', 'TAKE_DATE_TIME', 'HEADLINE_ALERT_TEXT',
       'ACCUMULATED_STORY_TEXT', 'TAKE_TEXT', 'PRODUCTS', 'TOPICS',
       'RELATED_RICS', 'NAMED_ITEMS', 'HEADLINE_SUBTYPE', 'STORY_TYPE',
       'TABULAR_FLAG', 'ATTRIBUTION', 'LANGUAGE'],
      dtype='object')
(3137, 20)


3137

### 1. Train a Doc2Vec Model quickly (can also use a pre-trained model from wiki for transfer-learning)

In [3]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=100)

# build vocabulary
model.build_vocab(train_corpus)

# train model
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

Wall time: 31.2 s


In [4]:
ranks = []
second_ranks = []

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    second_ranks.append(sims[1])
    

#### Some test of the model ability

In [30]:
# Random check to see similar documents are indeed identified (cltr-end for a few examples)
doc_id = np.random.randint(0, len(train_corpus) - 1)

# Compare and print the second-most-similar document
print('Selected Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Selected Document (476): «june infostrada sports result from the pirelli russian cup final match on saturday final saturday june cska moscow anzhi makhachkala halftime mins penalty shootout cska moscow win on penalties keywords soccer russia cup results»

Similar Document (855, 0.897797167301178): «june infostrada sports result from the romanian cup final match on saturday final saturday june petrolul ploiesti cfr cluj halftime keywords soccer romania cup results»



### 2. Cluster documents for topic identification using Hierachical Agglomerative Clustering
#### Here clustering is executed in each time window (4 hours as I choose). We are able to identify the top topic in each time window (my defiintion of "trending"). Such "onlineness" is a key feature of this approach.

In [20]:
def most_common(ls):
    return max(set(ls), key=ls.count)

def find_largest_topic(vector):
    
    cluster = AgglomerativeClustering(n_clusters= 140, linkage='ward')  # Here we want many clusters to get smaller ones
    cluster.fit_predict(vector)
    max_id = most_common(list(cluster.labels_))
    result_id = (cluster.labels_ == max_id)
    
    return result_id 

text_df['vector'] = [model.infer_vector(x.words) for x in train_corpus]

#### Break data into 4-hour window

In [21]:
result_df = text_df
result_df.HOUR = result_df.HOUR.apply(float)
n = int(result_df.HOUR.max() / 4)
result_df['Hour_4'] = pd.qcut(result_df.HOUR, n , labels = False)

In [22]:
topic_list =[]

for x in range(n):
    vector = result_df.loc[result_df.Hour_4 == x, 'vector'].tolist()
    top_topic_id = find_largest_topic(vector)
    top_docs = result_df.loc[result_df.Hour_4 == x, 'TAKE_TEXT'][top_topic_id]
    topic_list.append(top_docs)

In [23]:
# Not as clean as I would ideatlly like, 
# But similar document do get clustered together (e.g. market/economy and sports). 
# We may further fine-tune it.
topic_list[0] 

25                       [top, news, investment, banking]
151     [service, alert, msci, world, and, us, eod, de...
609     [exxon, mobil, xom, reports, unplanned, flarin...
610     [exxon, reports, unplanned, flaring, breakdown...
613                           [update, baseball, results]
1108                             [top, news, front, page]
1942                                    [emea, test, rcf]
2224                             [top, news, front, page]
2227                             [top, news, front, page]
2467                             [top, news, front, page]
3331    [motorcycling, motorcycling, grand, prix, moto...
4395    [service, alert, datascope, equities, planned,...
4932                                 [diary, japan, june]
5602                                     [diary, vietnam]
5607      [korea, may, manufacturing, pmi, vs, in, april]
5616               [buzz, eur, aud, uptrend, strong, for]
5622    [japan, corporate, capital, spending, falls, p...
6032    [north

### 3. Identify the "Names" in the largest topic using TF-IDF
#### Here alternatively we can retreive most frequent words that are in the largest topic but not the rest

In [24]:
# train a tfidf on entire corpus
tfidf = TfidfVectorizer(tokenizer=word_tokenize, stop_words='english', min_df=0.01, max_df=0.035)   
X_train = list(map(lambda x: ' '.join(word for word in x), result_df.TAKE_TEXT))
vect = tfidf.fit(X_train)
feature = np.array(vect.get_feature_names())
print(len(feature))

feature

899


array(['accessed', 'account', 'accused', 'action', 'activity', 'added',
       'adding', 'additional', 'affect', 'african', 'agency', 'ago',
       'agreed', 'agreement', 'agriculture', 'air', 'al', 'alex', 'allow',
       'allowed', 'amid', 'analysis', 'analyst', 'analysts', 'announced',
       'annual', 'anti', 'appeared', 'approved', 'apr', 'arab', 'area',
       'areas', 'argentina', 'asked', 'assets', 'atletico', 'attack',
       'aud', 'aug', 'australian', 'authorities', 'available', 'away',
       'ax', 'backs', 'bad', 'balance', 'bankers', 'barcelona', 'base',
       'baseball', 'basis', 'battle', 'bay', 'beat', 'beckons', 'began',
       'begin', 'beijing', 'believe', 'ben', 'benchmark', 'best',
       'better', 'biggest', 'bln', 'bo', 'board', 'boj', 'bombs', 'bond',
       'booms', 'boost', 'border', 'boston', 'brackets', 'brazil',
       'break', 'brien', 'bring', 'british', 'broader', 'budget',
       'building', 'bureau', 'business', 'buy', 'buying', 'buys', 'buzz',
     

In [25]:
#exaample
test = [x for y in topic_list[0] for x in y ]
test = ' '.join(test)
test
x = tfidf.transform([test])
y = x.toarray()[0].argsort()[-20:]
y

feature[y]

array(['outage', 'aud', 'equities', 'maintenance', 'pjm', 'spending',
       'corporate', 'falls', 'nadal', 'prix', 'shows', 'lme', 'vietnam',
       'reports', 'ca', 'emea', 'motorcycling', 'vs', 'diary', 'taiwan'],
      dtype='<U14')

In [26]:
# apply to each topic list to get most distinct words withitn the topic
def get_top_words(docs):
    
    # merge top topic into one document for tf-idf performance
    docs = [x for y in docs for x in y ] 
    docs =[' '.join(docs)]
    
    # get tf-idf and sort words by returned value
    tfidf = vect.transform(docs)
    sorted_tfidf_index = tfidf.toarray()[0].argsort()[-15:]
    df = pd.DataFrame(data = feature[sorted_tfidf_index], columns=['vocab'])
    
    return df

result = [get_top_words(x) for x in topic_list]

In [27]:
# save to dataframe
result = pd.DataFrame.from_dict({k : v['vocab'].tolist() for k, v in enumerate(result) })
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,spending,buzz,st,ns,ftseurofirst,fteu,buzz,freeport,fteu,fteu,frankfurt,future,gain,championship
1,corporate,standings,target,men,fuel,communications,baseball,fteu,ftseurofirst,ftseurofirst,fuel,frankfurt,futures,grand
2,falls,championship,weather,holding,funds,target,tokyo,ftseurofirst,fuel,fuel,funds,free,future,usa
3,nadal,sees,baseball,maintenance,future,outlook,white,funds,gains,funds,usd,freeport,free,post
4,prix,resistance,grand,singles,futures,weather,research,fuel,frankfurt,future,play,fteu,funds,doing
5,shows,mail,research,volume,gain,baseball,holding,trading,gain,futures,buzz,form,zone,prix
6,lme,italy,raises,malaysia,gains,qatar,rupees,italy,futures,gain,stronger,funds,ftseurofirst,la
7,vietnam,grand,prix,doing,game,research,raises,baseball,future,gains,result,fuel,fteu,planned
8,reports,rating,russia,prix,freeport,equities,holdings,grand,line,standings,cross,grand,fuel,maintenance
9,ca,moody,rating,inr,headline,swiss,shows,ax,coal,outlook,shows,italian,standings,liga
